In [3]:
import pandas

## Functions to get signal and single trial GLM

In [40]:
def get_signal(subject_id, run, mask):
    subject_id = '%02d' % int(subject_id)
    run = '%02d' % int(run)    
    run0 = int(run) - 1
    
    signal = np.loadtxt('/home/gdholla1/projects/bias/data/derivatives/extracted_signal/highpassed_motion/_mask_{mask}_subject_id_{subject_id}/_extracter_highpassed_motion{run0}/sub-{subject_id}_task-randomdotmotion_run-{run}_bold_unwarped_st_dtype_mcf_mask_gms_tempfilt_regfilt_maths_ts.txt'.format(**locals()))
    
    signal = (signal / signal.mean()) * 100
    signal -= signal.mean()
    
    return signal

def get_onsets(subject_id, run, design='single_trials_cue', shift=-3):

    df = pandas.read_pickle('/home/gdholla1/projects/bias/data/behavior/behavior.pandas')
    df = df[(df.subj_idx == subject_id) & (df.block == run)]

    df['onset_cue'] += shift
    df['onset_stim'] += shift
    
    
    if design == 'single_trials_cue':

        
        conditions = ['stim_easy', 'stim_hard']
        
        onsets = [df[df.difficulty == 'easy'].onset_stim.tolist(),
                  df[df.difficulty == 'hard'].onset_stim.tolist()]        

        

        conditions += ['trial %d' % trial for trial in df['trial_NR'].tolist()]
        
        onsets_cue = [[e] for e in df.onset_cue.tolist()]
        onsets +=  onsets_cue
        
    if design == 'single_trials_stim':

        
        conditions = ['payoff_cue', 'neutral_cue']
        
        onsets = [df[df.cue_validity != 'neutral'].onset_stim.tolist(),
                  df[df.cue_validity == 'neutral'].onset_stim.tolist()]        

        

        conditions += ['trial %d' % trial for trial in df['trial_NR'].tolist()]
        
        onsets_cue = [[e] for e in df.onset_stim.tolist()]
        onsets +=  onsets_cue
        
        
    
    return dict(zip(conditions, onsets))

# Get Single trials at cue

In [107]:
import itertools
import pyfir
from nipy.modalities.fmri import design_matrix, experimental_paradigm
import statsmodels.api as sm

level2_dict = []
adj_r = []
# pars = ['neutral_cue', 'left_cue', 'right_cue', 'neutral_easy_correct', 'valid_easy_correct', 'invalid_easy_correct', 'neutral_hard_correct', 'valid_hard_correct', 'invalid_hard_correct', 'easy_error', 'hard_error',]
# pars = 

mask = 'STh_L'

for subject_id in np.arange(1, 20):
    for run in [1,2,3]:

        try:
            onsets_dict = get_onsets(subject_id, run, 'single_trials_cue', shift=-3)
            pars = onsets_dict.keys()

            conditions = []
            onsets = []
            for key in onsets_dict.keys():
                conditions += [key] * len(onsets_dict[key])
                onsets += onsets_dict[key]

            paradigm =  experimental_paradigm.BlockParadigm(con_id=conditions, 
                                                            onset=onsets,
                                                            duration=[[1.]] * len(conditions))

            tr = 3.0
            frametimes = np.linspace(0, 387 * tr, 387)
            hrf_model = 'Canonical'

            X, names= design_matrix.dmtx_light(frametimes, paradigm, hrf_model=hrf_model, drift_model='polynomial', fir_delays=np.arange(11), drift_order=0)

            X /= X.max(0)
            X = pandas.DataFrame(X, columns=names)

            for i, mask in enumerate(['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C', 'STh_L', 'STh_R']):
                signal = get_signal(subject_id,  run, mask)
                r = sm.GLSAR(signal, X).iterative_fit()

                adj_r.append({'subject_id':subject_id, 'mask':mask, 'r2_adj':r.rsquared_adj, 'r2':r.rsquared, 'run':run})

                for par in pars:
                    level2_dict.append({'value':r.params[par],
                                        'par':par,
                                        'subject_id':subject_id,
                                        'mask':mask,
                                        'run':run})
        except Exception as e:
            print 'Error: %s/%s/%s' %  (subject_id, mask, e)

Error: 6/STh_R/all the input arrays must have same number of dimensions


In [108]:
behavior = pandas.read_pickle('/home/gdholla1/projects/bias/data/behavior/behavior.pandas')

In [109]:
df = pandas.DataFrame(level2_dict)


df['trial_NR'] = df.par.apply(lambda x: x.split()[-1] if x.split()[0] == 'trial' else np.nan)
df['mask'] = df['mask'].apply(lambda x: '%s_cue' % x)

df = df[~df.trial_NR.isnull()]
df['trial_NR'] = df.trial_NR.astype(int, inplace=True)

In [110]:
behavior['run'] = behavior.block
behavior['subject_id'] = behavior.subj_idx

In [111]:
df = behavior.merge(df.pivot_table(index=['subject_id', 'run', 'trial_NR'], columns=['mask'], values='value').reset_index(), on=['subject_id', 'run', 'trial_NR'])

# Get Single trials at stimulus

In [112]:
import itertools
import pyfir
from nipy.modalities.fmri import design_matrix, experimental_paradigm
import statsmodels.api as sm

level2_dict = []
adj_r = []
# pars = ['neutral_cue', 'left_cue', 'right_cue', 'neutral_easy_correct', 'valid_easy_correct', 'invalid_easy_correct', 'neutral_hard_correct', 'valid_hard_correct', 'invalid_hard_correct', 'easy_error', 'hard_error',]
# pars = 

mask = 'STh_L'

for subject_id in np.arange(1, 20):
    for run in [1,2,3]:

        try:
            onsets_dict = get_onsets(subject_id, run, 'single_trials_stim', shift=-3)
            pars = onsets_dict.keys()

            conditions = []
            onsets = []
            for key in onsets_dict.keys():
                conditions += [key] * len(onsets_dict[key])
                onsets += onsets_dict[key]

            paradigm =  experimental_paradigm.BlockParadigm(con_id=conditions, 
                                                            onset=onsets,
                                                            duration=[[1.]] * len(conditions))

            tr = 3.0
            frametimes = np.linspace(0, 387 * tr, 387)
            hrf_model = 'Canonical'

            X, names= design_matrix.dmtx_light(frametimes, paradigm, hrf_model=hrf_model, drift_model='polynomial', fir_delays=np.arange(11), drift_order=0)

            X /= X.max(0)
            X = pandas.DataFrame(X, columns=names)

            for i, mask in enumerate(['STh_L_A', 'STh_L_B', 'STh_L_C', 'STh_R_A', 'STh_R_B', 'STh_R_C', 'STh_L', 'STh_R']):
                signal = get_signal(subject_id,  run, mask)
                r = sm.GLSAR(signal, X).iterative_fit()

                adj_r.append({'subject_id':subject_id, 'mask':mask, 'r2_adj':r.rsquared_adj, 'r2':r.rsquared, 'run':run})

                for par in pars:
                    level2_dict.append({'value':r.params[par],
                                        'par':par,
                                        'subject_id':subject_id,
                                        'mask':mask,
                                        'run':run})
        except Exception as e:
            print 'Error: %s/%s/%s' %  (subject_id, mask, e)

/usr/local/lib/python2.7/dist-packages/nipy/modalities/fmri/design_matrix.py:218: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


Error: 6/STh_R/all the input arrays must have same number of dimensions


In [113]:
df_stim = pandas.DataFrame(level2_dict)


df_stim['trial_NR'] = df_stim.par.apply(lambda x: x.split()[-1] if x.split()[0] == 'trial' else np.nan)
df_stim['mask'] = df_stim['mask'].apply(lambda x: '%s_stim' % x)

df_stim = df_stim[~df_stim.trial_NR.isnull()]
df_stim['trial_NR'] = df_stim.trial_NR.astype(int, inplace=True)

In [114]:
df.merge(df_stim.pivot_table(index=['subject_id', 'run', 'trial_NR'], columns=['mask'], values='value').reset_index(), on=['subject_id', 'run', 'trial_NR'])

,block,trial_NR,cue_delay,stim_delay,cue,coherence,difficulty,stimulus,response,rt,...,STh_R_C_cue,STh_R_cue,STh_L_A_stim,STh_L_B_stim,STh_L_C_stim,STh_L_stim,STh_R_A_stim,STh_R_B_stim,STh_R_C_stim,STh_R_stim
0,1,1,750,1750,neutral,0.08,hard,left,-1,NaN,...,1.734490,0.602382,2.651964e+12,1.422970e+12,-1.459299e+12,1.236730e+12,3.295440e+11,3.279733e+12,2.506198e+12,2.395638e+12
1,1,2,0,1000,right,0.16,easy,left,2,720.0,...,-0.234407,0.185193,-5.718639e+12,-7.947347e+12,-3.635742e+12,-5.552347e+12,-4.911415e+12,-4.288434e+12,-4.065497e+12,-4.246169e+12
2,1,3,1500,1000,left,0.16,easy,left,1,486.0,...,-1.527146,-0.110078,-6.876488e+12,-9.556441e+12,-4.371868e+12,-6.676527e+12,-5.905826e+12,-5.156711e+12,-4.888635e+12,-5.105888e+12
3,1,4,750,1750,neutral,0.16,easy,left,2,689.0,...,-0.573978,-0.688172,3.216153e+12,1.725699e+12,-1.769755e+12,1.499836e+12,3.996524e+11,3.977476e+12,3.039376e+12,2.905296e+12
4,1,5,1500,1000,neutral,0.16,easy,left,1,1339.0,...,-2.142813,-0.942222,3.215691e+12,1.725451e+12,-1.769501e+12,1.499621e+12,3.995950e+11,3.976905e+12,3.038939e+12,2.904878e+12
5,1,6,1500,3250,neutral,0.08,hard,left,1,1312.0,...,-3.072903,-2.397118,2.808772e+12,1.507109e+12,-1.545585e+12,1.309856e+12,3.490295e+11,3.473660e+12,2.654386e+12,2.537290e+12
6,1,7,2250,3250,neutral,0.16,easy,left,1,1429.0,...,-2.309920,-2.308500,2.722602e+12,1.460872e+12,-1.498168e+12,1.269671e+12,3.383217e+11,3.367092e+12,2.572952e+12,2.459448e+12
7,1,8,1500,2500,left,0.16,easy,right,1,424.0,...,-0.300276,-0.590525,-5.563640e+12,-7.731940e+12,-3.537198e+12,-5.401855e+12,-4.778295e+12,-4.172200e+12,-3.955304e+12,-4.131080e+12
8,1,9,1500,1750,left,0.16,easy,left,1,923.0,...,1.899101,0.783075,-4.932073e+12,-6.854235e+12,-3.135666e+12,-4.788653e+12,-4.235878e+12,-3.698585e+12,-3.506311e+12,-3.662133e+12
9,1,10,2250,2500,neutral,0.16,easy,left,1,1092.0,...,-0.423844,-0.473230,2.809097e+12,1.507284e+12,-1.545764e+12,1.310008e+12,3.490700e+11,3.474062e+12,2.654694e+12,2.537584e+12


In [115]:
X

,neutral_cue,payoff_cue,trial 1,trial 10,trial 100,trial 101,trial 102,trial 103,trial 104,trial 105,...,trial 91,trial 92,trial 93,trial 94,trial 95,trial 96,trial 97,trial 98,trial 99,constant
0,9.975281e-16,3.590583e-16,-1.366117e-15,-1.910079e-15,-4.416114e-15,-4.442053e-15,-4.446796e-15,-4.346197e-15,-1.480637e-15,-1.815132e-15,...,-2.150912e-15,-1.978392e-15,-1.894960e-15,-1.768946e-15,-2.081848e-15,-4.148906e-15,-3.477241e-15,-1.644231e-15,-4.389996e-15,1.0
1,-3.515463e-15,-7.662639e-15,7.035101e-15,6.950328e-15,1.200988e-15,1.222376e-15,1.323626e-15,1.156381e-15,6.197001e-15,6.881206e-15,...,8.278180e-15,7.176862e-15,7.288841e-15,6.850003e-15,7.256419e-15,1.430609e-15,9.088256e-16,7.224736e-15,1.262352e-15,1.0
2,-2.937580e-15,-7.723991e-16,-7.758496e-16,-1.423197e-15,5.333547e-16,4.673183e-16,6.772877e-16,3.392057e-16,-8.564638e-16,-1.271997e-15,...,-1.508376e-15,-1.387049e-15,-1.246147e-15,-1.187782e-15,-1.479237e-15,6.294804e-16,4.226188e-16,-1.118212e-15,4.753116e-16,1.0
3,-3.498445e-15,-1.183767e-15,-2.881820e-16,-9.114641e-16,1.255317e-15,1.203802e-15,1.251325e-15,1.166528e-15,-4.589491e-16,-7.887947e-16,...,-9.164096e-16,-8.701264e-16,-6.971018e-16,-6.891272e-16,-9.448629e-16,1.382407e-15,1.007036e-15,-6.290032e-16,1.237981e-15,1.0
4,-1.253363e-15,2.199709e-01,2.601052e-01,-1.461255e-17,-6.179509e-16,-4.211587e-16,-2.961146e-16,-3.214485e-16,-5.228602e-18,5.963945e-17,...,-5.326798e-17,-2.415028e-16,-1.376883e-16,-9.563851e-17,-3.634501e-16,-3.577567e-16,-1.893099e-16,-2.024897e-16,-4.665953e-16,1.0
5,1.128556e-15,8.456998e-01,1.000000e+00,-1.227232e-16,3.875804e-17,-3.013879e-16,4.644305e-17,2.998736e-16,-1.005496e-15,-3.127857e-16,...,-8.356269e-16,-1.323565e-15,-1.278244e-15,-1.042792e-15,-1.555711e-15,-4.982056e-16,3.302757e-16,-1.879260e-15,-4.515884e-16,1.0
6,-1.488674e-15,3.829470e-01,4.528167e-01,-1.276461e-16,7.579743e-16,6.377595e-16,7.028783e-16,1.032508e-15,-2.927758e-16,-1.139979e-16,...,-3.669456e-16,-6.236716e-16,-5.447050e-16,-4.471785e-16,-7.936138e-16,6.098521e-16,7.401135e-16,-7.333137e-16,5.637475e-16,1.0
7,-1.746182e-15,1.481878e-01,-8.459457e-02,-6.114923e-16,-2.759889e-16,-2.232596e-16,-2.272836e-16,-2.417286e-16,-2.775572e-16,-4.590880e-16,...,-4.155298e-16,-3.592336e-16,-4.283656e-16,-3.673844e-16,-5.406880e-16,-7.627647e-17,-1.613787e-16,-2.094943e-16,-8.739936e-17,1.0
8,6.045386e-16,7.001808e-01,-1.720634e-01,-1.596684e-15,-2.412807e-16,9.730120e-17,-7.710573e-17,-3.278012e-16,-1.031188e-15,-1.269876e-15,...,-1.298899e-15,-1.048989e-15,-1.787849e-15,-1.449969e-15,-1.508511e-15,-3.982070e-17,-1.021016e-16,-1.280142e-15,3.575180e-16,1.0
9,-8.096635e-16,4.071077e-01,-9.408297e-02,-1.292210e-16,4.108700e-18,9.647472e-17,9.744265e-17,-6.075526e-17,1.092902e-16,9.209979e-18,...,2.247742e-16,2.060660e-16,-3.348898e-17,7.581710e-17,5.004902e-19,1.710046e-16,5.873448e-17,1.381544e-16,2.421162e-16,1.0
